# Threshold images

K.Garner, 2020

This wf takes an spmT image, anatomically masks it. The remaining is saved.
Designed to be repeated over t's and over masks.

In [1]:
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
import nipype.interfaces.fsl.maths as fsl
from nipype.interfaces import spm as spm
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html

matlab_cmd = '/opt/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

210308-19:21:14,699 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
thrsh = pe.Workflow(name="thrsh") # workflow to run the analysis

## Data grabbing and sinking

In [3]:
dg = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                         outfields=['ts','msks']), name='data-grabber')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'ts': [['sub', 'TR']],
                           'msks': [['sub', 'TR']]}
dg.inputs.field_template = {'msks': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/TR%s/MASKS/*.nii.gz',
                            'ts': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-%s/TR%s/FLGLM/spmT*.nii'}

In [4]:
# # debugging
# dg.inputs.sub = '01'
# dg.inputs.TR = '1510'
# res = dg.run()
# res.outputs

In [5]:
def printSubPath(fullFilePath):
    # function to split filepath into constituent parts, then print string to add as input to DataSink for the container string
    # given the full filepath, this extracts the subject folder and TR strings for input
    # into DataSink
    import os
    import re
    fname = os.path.normpath(fullFilePath[0])
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0], [t for t in l if re.search('TR([0-9])', t)][0]
    name = '/'.join(name)
    name
    return name

In [6]:
#printSubPath(res.outputs.msks)

In [7]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_thrsh-t([0-9]*)', ''),
                 ('_absthat([0-9]*)', ''),
                 ('_mask-t([0-9])*', ''),
                 ('_multi([0-9])*', '')] 
ds.inputs.regexp_substitutions = substitutions

In [8]:
datinf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                 name='dat-info')
#datinf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
datinf.iterables = [('sub', ['05']), ('TR', ['700'])]
datinf.iterables 

[('sub', ['05']), ('TR', ['700'])]

## Gzip interface

get .nii -> .nii.gz

In [9]:
from nipype.interfaces.base import (
    TraitedSpec,
    CommandLineInputSpec,
    CommandLine,
    File
)
import os

class GZipInputSpec(CommandLineInputSpec):
    in_file = File(desc="File", exists=True, mandatory=True, argstr="%s")

class GZipOutputSpec(TraitedSpec):
    out_file = File(desc = "Zip file", exists = True)

class GZipTask(CommandLine):
    input_spec = GZipInputSpec
    output_spec = GZipOutputSpec
    _cmd = 'gzip'

    def _list_outputs(self):
            outputs = self.output_spec().get()
            outputs['out_file'] = os.path.abspath(self.inputs.in_file + ".gz")
            return outputs
        
gzip = pe.MapNode(GZipTask(), iterfield='in_file', name='gzip')

## functions for list cats

In [10]:
def tcat(fullFileList, nMasks=9):
    return fullFileList*nMasks

In [11]:
def maskcat(fullFileList, nCon=7):
    import numpy as np
    return np.ndarray.tolist(np.repeat(fullFileList, nCon))

## define output names for t x msk

In [12]:
tnms = ['spmT_0001', 'spmT_0002', 'spmT_0003', 'spmT_0005', 'spmT_0006', 'spmT_0007', 'spmT_0009']
msks = ['rCN', 'rFEF', 'rGPe', 'rGPi', 'rIPS', 'rLOC', 'rPut', 'rSTN', 'rVS']
touts = ['th'+i+'_'+n+'.nii.gz' for i in msks for n in tnms]

In [13]:
touts

['thrCN_spmT_0001.nii.gz',
 'thrCN_spmT_0002.nii.gz',
 'thrCN_spmT_0003.nii.gz',
 'thrCN_spmT_0005.nii.gz',
 'thrCN_spmT_0006.nii.gz',
 'thrCN_spmT_0007.nii.gz',
 'thrCN_spmT_0009.nii.gz',
 'thrFEF_spmT_0001.nii.gz',
 'thrFEF_spmT_0002.nii.gz',
 'thrFEF_spmT_0003.nii.gz',
 'thrFEF_spmT_0005.nii.gz',
 'thrFEF_spmT_0006.nii.gz',
 'thrFEF_spmT_0007.nii.gz',
 'thrFEF_spmT_0009.nii.gz',
 'thrGPe_spmT_0001.nii.gz',
 'thrGPe_spmT_0002.nii.gz',
 'thrGPe_spmT_0003.nii.gz',
 'thrGPe_spmT_0005.nii.gz',
 'thrGPe_spmT_0006.nii.gz',
 'thrGPe_spmT_0007.nii.gz',
 'thrGPe_spmT_0009.nii.gz',
 'thrGPi_spmT_0001.nii.gz',
 'thrGPi_spmT_0002.nii.gz',
 'thrGPi_spmT_0003.nii.gz',
 'thrGPi_spmT_0005.nii.gz',
 'thrGPi_spmT_0006.nii.gz',
 'thrGPi_spmT_0007.nii.gz',
 'thrGPi_spmT_0009.nii.gz',
 'thrIPS_spmT_0001.nii.gz',
 'thrIPS_spmT_0002.nii.gz',
 'thrIPS_spmT_0003.nii.gz',
 'thrIPS_spmT_0005.nii.gz',
 'thrIPS_spmT_0006.nii.gz',
 'thrIPS_spmT_0007.nii.gz',
 'thrIPS_spmT_0009.nii.gz',
 'thrLOC_spmT_0001.nii.gz',

## FSL functionality to apply anatomical mask

In [14]:
# maskt = pe.MapNode(fsl.ApplyMask(), iterfield = ['in_file', 'mask_file', 'out_file'], name='mask-t')
# maskt.inputs.out_file = touts
# maskt.inputs.nan2zeros = True

maskt = pe.MapNode(fsl.BinaryMaths(), iterfield = ['in_file', 'operand_file', 'out_file'], name='mask-t')
maskt.inputs.operation = 'mul'
maskt.inputs.nan2zeros = True
maskt.inputs.out_file = touts

## to take the absolute value of the image

In [15]:
# absv = pe.MapNode(fsl.UnaryMaths(), iterfield = ['in_file', 'out_file'], name='absthat')
# absv.inputs.operation = 'abs'
# absv.out_file = touts

# rounding test

In [16]:
# multIm = pe.MapNode(fsl.BinaryMaths(), iterfield = ['in_file', 'out_file'], name='multi')
# multIm.inputs.operand_value = 10
# multIm.inputs.operation = 'mul'
# multIm.inputs.out_file = touts

## and to apply threshold

In [17]:
# thrsht = pe.MapNode(fsl.Threshold(), iterfield = ['in_file', 'out_file'], name='thrsh-t')
# thrsht.inputs.thresh = 2.326 # p=.01 - 2 tailed
# thrsht.inputs.out_file = touts
#thrsht.inputs.internal_datatype = 'input'
# thrsht.inputs.use_robust_range = True
# thrsht.inputs.use_nonzero_voxels = True

## Connect workflow

In [18]:
# thrsh.connect([(datinf, dg, [('sub', 'sub')]),
#               (datinf, dg, [('TR', 'TR')]),
#               (dg, ds, [(('msks', printSubPath), 'container')]),
# #               (dg, gzip, [('ts', 'in_file')]), # zip the t-images # use these 2 lines if .nii -> .nii.gz is required
# #               (gzip, maskt, [(('out_file', tcat), 'in_file')]),
#               (dg, maskt, [(('ts', tcat), 'in_file')]), 
#               (dg, maskt, [(('msks', maskcat), 'mask_file')]),
#               (maskt, absv, [('out_file', 'in_file')]), 
#               (absv, thrsht, [('out_file', 'in_file')]),
#               (thrsht, ds, [('out_file', 'THRSH')])
# ])
# res = thrsh.run()

## Debugging workflow

In [19]:
thrsh.connect([(datinf, dg, [('sub', 'sub')]),
               (datinf, dg, [('TR', 'TR')]),
               (dg, ds, [(('msks', printSubPath), 'container')]),
               (dg, maskt, [(('ts', tcat), 'in_file')]), 
               (dg, maskt, [(('msks', maskcat), 'operand_file')]),
               (maskt, ds, [('out_file', 'THRSH')])
])
res = thrsh.run()

210308-19:21:17,728 nipype.workflow INFO:
	 Workflow thrsh settings: ['check', 'execution', 'logging', 'monitoring']
210308-19:21:17,741 nipype.workflow INFO:
	 Running serially.
210308-19:21:17,742 nipype.workflow INFO:
	 [Node] Setting-up "thrsh.data-grabber" in "/tmp/tmpfwkhbcyi/thrsh/_TR_700_sub_05/data-grabber".
210308-19:21:17,748 nipype.workflow INFO:
	 [Node] Running "data-grabber" ("nipype.interfaces.io.DataGrabber")
210308-19:21:17,755 nipype.workflow INFO:
	 [Node] Finished "thrsh.data-grabber".
210308-19:21:17,757 nipype.workflow INFO:
	 [Node] Setting-up "thrsh.mask-t" in "/tmp/tmpjna8sb3n/thrsh/_TR_700_sub_05/mask-t".
210308-19:21:17,783 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t0" in "/tmp/tmpjna8sb3n/thrsh/_TR_700_sub_05/mask-t/mapflow/_mask-t0".
210308-19:21:17,787 nipype.workflow INFO:
	 [Node] Running "_mask-t0" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-0

210308-19:21:20,396 nipype.workflow INFO:
	 [Node] Running "_mask-t13" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/FLGLM/spmT_0009.nii -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR700/MASKS/rFEF_trans.nii.gz thrFEF_spmT_0009.nii.gz
210308-19:21:20,587 nipype.workflow INFO:
	 [Node] Finished "_mask-t13".
210308-19:21:20,590 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t14" in "/tmp/tmpjna8sb3n/thrsh/_TR_700_sub_05/mask-t/mapflow/_mask-t14".
210308-19:21:20,595 nipype.workflow INFO:
	 [Node] Running "_mask-t14" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/FLGLM/spmT_0001.nii -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR700/MASKS/rGPe_trans.nii.gz thrGPe_spmT_0001.nii.gz
210308-19:21:20,797 nipype.workflow INFO:
	 [Node] Fin

210308-19:21:23,271 nipype.workflow INFO:
	 [Node] Finished "_mask-t27".
210308-19:21:23,273 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t28" in "/tmp/tmpjna8sb3n/thrsh/_TR_700_sub_05/mask-t/mapflow/_mask-t28".
210308-19:21:23,277 nipype.workflow INFO:
	 [Node] Running "_mask-t28" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/FLGLM/spmT_0001.nii -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR700/MASKS/rIPS_trans.nii.gz thrIPS_spmT_0001.nii.gz
210308-19:21:23,473 nipype.workflow INFO:
	 [Node] Finished "_mask-t28".
210308-19:21:23,475 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t29" in "/tmp/tmpjna8sb3n/thrsh/_TR_700_sub_05/mask-t/mapflow/_mask-t29".
210308-19:21:23,480 nipype.workflow INFO:
	 [Node] Running "_mask-t29" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivative

210308-19:21:25,935 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t42" in "/tmp/tmpjna8sb3n/thrsh/_TR_700_sub_05/mask-t/mapflow/_mask-t42".
210308-19:21:25,938 nipype.workflow INFO:
	 [Node] Running "_mask-t42" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/FLGLM/spmT_0001.nii -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR700/MASKS/rPut_trans.nii.gz thrPut_spmT_0001.nii.gz
210308-19:21:26,112 nipype.workflow INFO:
	 [Node] Finished "_mask-t42".
210308-19:21:26,115 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t43" in "/tmp/tmpjna8sb3n/thrsh/_TR_700_sub_05/mask-t/mapflow/_mask-t43".
210308-19:21:26,120 nipype.workflow INFO:
	 [Node] Running "_mask-t43" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/FLGLM/spmT_0002.nii -nan -mul /scratch/qbi/uqkga

210308-19:21:28,537 nipype.workflow INFO:
	 [Node] Running "_mask-t56" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/FLGLM/spmT_0001.nii -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR700/MASKS/rVS_trans.nii.gz thrVS_spmT_0001.nii.gz
210308-19:21:28,718 nipype.workflow INFO:
	 [Node] Finished "_mask-t56".
210308-19:21:28,723 nipype.workflow INFO:
	 [Node] Setting-up "_mask-t57" in "/tmp/tmpjna8sb3n/thrsh/_TR_700_sub_05/mask-t/mapflow/_mask-t57".
210308-19:21:28,728 nipype.workflow INFO:
	 [Node] Running "_mask-t57" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/FLGLM/spmT_0002.nii -nan -mul /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR700/MASKS/rVS_trans.nii.gz thrVS_spmT_0002.nii.gz
210308-19:21:28,913 nipype.workflow INFO:
	 [Node] Finishe

210308-19:21:29,943 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/THRSH/_TR_700_sub_05/_mask-t15/thrGPe_spmT_0002.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/THRSH///thrGPe_spmT_0002.nii.gz
210308-19:21:29,945 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/THRSH/_TR_700_sub_05/_mask-t16/thrGPe_spmT_0003.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/THRSH///thrGPe_spmT_0003.nii.gz
210308-19:21:29,947 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/THRSH/_TR_700_sub_05/_mask-t17/thrGPe_spmT_0005.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/THRSH///thrGPe_spmT_0005.nii.gz
210308-19:21:29,949 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/THRSH/_TR_700_sub_05/_mask-t18/thrGPe_spmT_0006.nii.gz -> /scratch/qbi/uqkg

210308-19:21:30,11 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/THRSH/_TR_700_sub_05/_mask-t46/thrPut_spmT_0006.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/THRSH///thrPut_spmT_0006.nii.gz
210308-19:21:30,13 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/THRSH/_TR_700_sub_05/_mask-t47/thrPut_spmT_0007.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/THRSH///thrPut_spmT_0007.nii.gz
210308-19:21:30,15 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/THRSH/_TR_700_sub_05/_mask-t48/thrPut_spmT_0009.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/THRSH///thrPut_spmT_0009.nii.gz
210308-19:21:30,19 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/THRSH/_TR_700_sub_05/_mask-t49/thrSTN_spmT_0001.nii.gz -> /scratch/qbi/uqkgarn1